docker pull python:3.11-slim<br>
:tag-version

list images <br>
```docker images```<br>
```docker rmi <image_id>```<br>
```docker -f rmi <image_id>``` forced

### Running a Container Interactively
Let's run a container from this image. We'll use the -it flag to get an interactive terminal session inside the container. The command we'll run inside it is bash.<br>
```docker run -it python:3.11-slim bash```<br>
Your terminal prompt should change. It will look something like root@a1b2c3d4e5f6:/#. You are now inside the container!

Try running these commands inside the container to explore its isolated environment:
<br>
ls -la (Lists files in the current directory)
<br>
pwd (Shows your current directory, which is /)
<br>
python --version (Confirms the Python version inside the container)
<br>
To leave the container, simply type exit and press Enter.

## Managing Containers
When you exited, the container stopped. Let's see how we can manage it.
<br>
List running containers:

```
docker ps
```
This will show an empty list, because your container is no longer running.
<br>
List all containers (running and stopped):
<br>
```
docker ps -a
```
<br>
Now you'll see your container listed with a status like Exited (0) .... Note its CONTAINER ID.
<br>
Remove a stopped container:
It's good practice to clean up containers you no longer need. Copy the first few characters of the CONTAINER ID from the previous command and use them here:<br>
```docker rm a1b2c3d4e5f6```
Now if you run docker ps -a again, the container will be gone.

Pro Tip: To remove all stopped containers at once, you can run 
<br>
```docker container prune```

## Build Your Custom Image
Open your terminal and make sure you are in the project directory you just created. Run the docker build command:
```docker build -t first-app .```

### Run Your Application Container
```docker run -p 5001:5000 my-first-app```

## Data persistence
Excellent. Let's tackle the final lesson of our first module: Module 1, Lesson 4: Data Persistence with Volumes.<br>

This lesson addresses a critical question: If containers are temporary, where do I keep my permanent data?<br>

Theory: Don't Lose Your Data! 💾<br>
The Problem: The Ephemeral Filesystem<br>
As we discussed, a container's writable layer is ephemeral. When you run docker rm to delete a container, that top layer is destroyed. Any data your application generated—like log files, user uploads, or database entries—is permanently lost.
<br>
This is fine for stateless applications, but it's a deal-breaker for anything that needs to remember information. We need a way to store data outside of the container's lifecycle. Docker provides two main ways to do this.<br>

### Solution 1: Bind Mounts (A "Window" to Your Host)
**Primary Use Case: Development.**<br> It's perfect for live code reloading. You can edit your Python code on your host machine with your favorite code editor, and the changes are immediately reflected inside the running container without needing to rebuild the image.

```docker build -t my-first-app .```<br>
```docker run -p 5001:5000 -v "%cd%":/app my-first-app``` or ```docker run -p 5001:5000 -v "$(pwd)":/app my-first-app```<br>


Test Live Reloading:

Go to http://localhost:5001 in your browser. You'll see the original message.

Now, without stopping the container, go back to your code editor and change the message in app.py. For example: return 'Live Reload is Awesome!'. Save the file.

Refresh your browser. The new message will appear instantly! You've just used a bind mount to sync your local code with the running container.

### Solution 2: Volumes (A "Managed Hard Drive")<br>
A volume is a dedicated storage area that is fully managed by Docker. You tell Docker to attach a volume to your container, and Docker handles the details of where to store it on the host machine. The key is that the volume's lifecycle is completely separate from any container's lifecycle.<br>

Analogy: Think of it as plugging a dedicated, managed external hard drive into your container. You can unplug it from one container and plug it into another, and the data remains intact.<br>

Primary Use Case: Production & Storing Application Data. This is the preferred way to handle data you care about, such as database files, configuration, logs, and user-generated content. They are more portable and platform-agnostic than bind mounts.
<br>
Structure: ```-v volume-name:/path/in/container```

```docker build -t my-logger .``` <br>
```docker run -d --name logger-container -v log-data:/data/logs my-logger```
**The -d flag runs the container in "detached" mode (in the background).**

```docker stop logger-container```<br>
```docker rm logger-container```

```docker run --rm -v log-data:/data/logs busybox cat /data/logs/app.log```<br>
see if the volume we created we can mount it and run in another container

```docker volume ls```            # See your 'log-data' volume<br>
```docker volume rm log-data```       # Remove the volume